# Testing and Benchmarking

We load the best weights we've found, and load them in a model.

In [36]:
# Load best model weights
from keras.models import load_model
model = load_model('tmp/weights-39ep-16x106.hdf5')

Unlike training we resize the images online here, it takes about 10 minutes.

In [37]:
import glob
from keras.preprocessing.image import img_to_array
from skimage.transform import resize
from skimage.io import imread

def preprocess_image(fname, crop=212, downsample=106):
    sample_img = imread(fname)
    crop_from = int(crop / 2)
    crop_to   = int(crop * 1.5)
    sample_img = sample_img.T[:, crop_from:crop_to, crop_from:crop_to]
    sample_img = resize(sample_img, (3, downsample, downsample), mode='reflect')
    return sample_img

def fetch_test_images():
    while 1:
        for fname in test_files:
            image = preprocess_image(fname)
            image = img_to_array(image)
            test = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            yield (test)

In [ ]:
test_files = glob.glob('../data/images_test_rev1/*.jpg')

Generate predictions.

In [ ]:
predictions = model.predict_generator(fetch_test_images(),
                        steps=len(test_files),
                        max_queue_size=32,)
predictions.shape # assert (79975, 37)

Record all results into a csv file with the id and the predictions. The CSV file will be about 40MB big. We only load the all_zeros.csv file for its header.

In [ ]:
from os import path

header = open('../data/all_zeros_benchmark.csv','r').readlines()[0]

with open('submission_1.csv','w') as outfile:
    outfile.write(header)
    for i in range(len(test_files)):
        fname = test_files[i]
        file_id = path.split(fname)[-1] \
                      .split('.')[0]
        pred = predictions[i]
        outline = file_id + "," + ",".join([str(x) for x in pred])
        outfile.write(outline + "\n")